In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
    verbose=True,
)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

def invoke_chain(chain, question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    return result

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

examples = [
    {
        "question": "탑건",
        "답변": "🛩️👨‍✈️🔥",
    },
    {
        "question": "대부",
        "답변": "👨‍👨‍👦🔫🍝",
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{답변}"),
])
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화를 이모지 3개로 표현하는 전문가입니다. 기존 답변과 같은 포맷으로 답변해주세요."),
    MessagesPlaceholder(variable_name="history"),
    example_prompt,
    ("human", "{question}")
])

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

In [3]:
invoke_chain(chain, "기생충")

AIMessage(content='🏠💰🎭')

In [4]:
invoke_chain(chain, "스파이더맨")

AIMessage(content='🕷️🕸️🕺')

In [5]:
invoke_chain(chain, "내가 처음에 뭘 물어봤나요?")

AIMessage(content='"기생충"이라는 영화를 이모지 3개로 표현해달라고 물어보셨습니다.')

In [6]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='기생충'),
  AIMessage(content='🏠💰🎭'),
  HumanMessage(content='스파이더맨'),
  AIMessage(content='🕷️🕸️🕺'),
  HumanMessage(content='내가 처음에 뭘 물어봤나요?'),
  AIMessage(content='"기생충"이라는 영화를 이모지 3개로 표현해달라고 물어보셨습니다.')]}